# Interpolation demonstration
Demonstrates the process of interpolation from both time and frequency domains.

### Preamble
Start by importing the Python libraries that we will require

In [ ]:
import numpy as np
from matplotlib.ticker import FuncFormatter
from IPython import display
import pylab as pl

### Define the signal
This is defined in the frequency domain as a rectangular pulse centred on zero frequency (d.c.).  Due to the cyclic nature of the FFT implementation, the pulse appears at the start and the end of the frequency record.

In [ ]:
Xw = np.zeros(65536)
Freq = np.zeros(65536)
Xw[0:8] = np.ones(8)
Xw[65529:65536] = np.ones(7)

### Generate the corresponding samples in the time domain
This is simply achieved with an inverse discrete Fourier transform.

In [ ]:
Wt = np.fft.ifft(Xw)

### Plotting Functions
The time domain plot requires fixed y-axis labels to ensure that the size doesn't change when stepping through the animation.

In [ ]:
def plot_timeDomain(Wt, num_ticks):
    """
       Plot the time domain.
       
       INPUT:
           Wt  (array-like): Its real part is the vertical coordinates of the data points. 
           num_ticks  (int): Number of tick marks to show.
    """
    # subpplot
    ax1 = pl.subplot(2,1,1)
    pl.plot(Wt.real)
   
    # Set axis limits 
    pl.axis([0, 8192, -Wt[0].real*0.5, Wt[0].real*1.05])
    
    # Set y axis format 
    # Lambda function to define ylabel format
    # FuncFormatter function to define function to set the label
    formatter = FuncFormatter(lambda x, pos:"%.4f"%x)
    # and apply this to the label
    ax1.yaxis.set_major_formatter(formatter)
    
    # Set the current tick locations and labels of the x-axis. 
    pl.xticks(np.linspace(0, 8192, num_ticks), np.arange(0, num_ticks))
    
    pl.title('Time domain')

In [ ]:
def plot_freqDomain(Freq, tickLabel):
    """
       Plot the frequency domain.
       
       INPUT:
           Freq      (array-like): The vertical coordinates of the data points. 
           tickLabel (array-like): The tick label of x axis.
    """
    ax2 = pl.subplot(2,1,2)
    pl.plot(Freq)
    
    pl.axis([31500, 34036, 0, 1.2])
    pl.xticks(np.linspace(31745, 33793, 9), tickLabel)
              
    pl.title('Frequency domain')

### Demonstrate the effect of sampling and interpolation
First, the duality between sampling in the time domain and periodicity in the frequency domain is demonstrated.  This is done by successively adding periodic terms in the frequency domain, until the frequency domain is fully periodic.  At that point the time domain becomes sampled.

In the next step, the time domain is upsampled with the addition of zero values between each sample.  In the frequency domain this changes the location of the sampling frequency, $f_s$.

Now that the sampling frequency is higher than the first periodic repetition, then the periodic repetition at the frequency of $f_s/2$ can be removed.  In the animation it is progressively scaled down until it is zero.  The effect of this in the time domain can be seen as it modifies the intermediate samples until they attain values that "smooth" the final resulting sampled sequence.

In the figure, the time domain is in the upper plot, and the frequency domain is in the lower plot.

Enlarge the figure size and label font size

In [ ]:
pl.rcParams["figure.figsize"] = [16,8]
pl.rcParams.update({'font.size': 13})

In [ ]:
#### time interval is 512 
# showing the time domain in the upper plot
plot_timeDomain(Wt = Wt, num_ticks = 17)

# And plot it 
x_ticklabel = ['-8fs', '-6fs', '-4fs', '-2fs', '0', '2fs', '4fs', '6fs', '8fs']
plot_freqDomain(np.roll(Xw, 32768), x_ticklabel)

# Adjusts params so that the subplots fit in to the figure area. 
# pl.tight_layout()
pl.show()

# Wait for user input before moving on to the next stage 
input('Paused.  Press enter key to continue...')

# In the next loop, successively add periodic repetitions of the pulse,
# performing the same computations
for i in np.arange(128, 32768, 128):
    Xw[i-7:i+8] = np.ones(15)
    Xw[65529-i:65544-i] = np.ones(15)
    if (i<2048) | (i % 2048 == 1920):
        Wt = np.fft.ifft(Xw)
        plot_timeDomain(Wt = Wt, num_ticks = 17)
    
        #Freq[0:32768] = Xw[32768:65536]
        #Freq[32768:65536] = Xw[0:32768]
        plot_freqDomain(np.roll(Xw, 32768), x_ticklabel)
    
        # For the first few, wait for user input - useful in a lecture.
        # For the remainder, just pause for a very short time
        display.clear_output(wait=True)
    
        pl.show()
        if (i<512):
            input('Paused.  Press enter key to continue...')        
        
#### time interval is 256       
# Again, wait for a keystroke from the user
input('Paused.  Press enter key to continue...')
display.clear_output(wait=True)

# Show a rescaled x-axis indicating insertion of zero samples
plot_timeDomain(Wt = Wt, num_ticks = 33)

XTickLabel =['-4fs', '-3fs', '-2fs', '-fs', '0', 'fs', '2fs', '3fs', '4fs']
plot_freqDomain(np.roll(Xw, 32768), XTickLabel)

pl.show()

input('Paused.  Press enter key to continue...')

# Define the mask we will use for attenuating alternate repetitions 
# of the frequency domain
Mask = np.zeros(65536)
for i in np.arange(128, 32768, 256):
    Mask[i-7:i+8] = np.ones(15)
    Mask[65529-i:65544-i] = np.ones(15)

# Successively attenuate every other periodic repetition in frequency,
# using the mask defined, and recompute the time domain samples
for i in range(25):
    Xw = Xw - 0.04*Mask
    Wt = np.fft.ifft(Xw)
    plot_timeDomain(Wt = Wt, num_ticks = 33)
    
    plot_freqDomain(np.roll(Xw, 32768), XTickLabel)

    display.clear_output(wait=True)
    pl.show()

© The University of Edinburgh: Produced by D. Laurenson, School of Engineering. Initial code conversion by Xing Zixiao.